In [1]:

import folium
from folium.plugins import MarkerCluster
import pandas_gbq
import os

Read latitude, longitude of sales data from BigQuery

In [2]:
sql_map = '''select latitude, longitude from `[dataset-id].transaction_history`'''

df = pandas_gbq.read_gbq(sql_map, project_id="[project-id]")

Downloading: 100%|██████████|


In [3]:
df.shape

(6383713, 2)

**Preprocessing** - dropping null rows and dropping duplicate rows

In [4]:

df = df.dropna(subset=['latitude', 'longitude'])
df = df.drop_duplicates()
df.shape

(2041678, 2)

**Preprocessing** <br>
There are some outliers (coordinates from other countries). I am using a threshold (+-2 standard deviation from mean) to remove these outliers

In [5]:
mean_lat  = df['latitude'].mean()
std_lat = df['latitude'].std()

mean_long  = df['longitude'].mean()
std_long = df['longitude'].std()


In [6]:
print(mean_lat,mean_long)
print(std_lat,std_long)

23.86016475213528 90.2464927868465
0.773521164350342 0.9517392840173517


In [7]:
threshold_lat = 2 * std_lat
threshold_lon = 2 * std_long

# Filter the dataset based on the threshold
filtered_df = df[
    (df['latitude'] >= mean_lat - threshold_lat) & (df['latitude'] <= mean_lat + threshold_lat) &
    (df['longitude'] >= mean_long - threshold_lon) & (df['longitude'] <= mean_long + threshold_lon)
]
filtered_df.shape

(1951319, 2)

In total , there are 2 million data. Since I dont have the computational power to work with 2 million data, I will take a sample of 700,000 data points

In [8]:
gdf = df.sample(frac=0.35)
gdf.shape

(714587, 2)

Initializing the folium map, the location parameter specifies the center coordinates of the map which is near Dhaka, Bangladesh

In [10]:

m = folium.Map(location=[23.8041, 90.4152], zoom_start=7)

Adding each coordinates to the folium map

In [11]:
marker_cluster = MarkerCluster().add_to(m)

# Add your data points to the cluster
for idx, row in gdf.iterrows():
    folium.Marker([row['latitude'], row['longitude']]).add_to(marker_cluster)

In [ ]:
m

Save the map as an interactive HTML file. When you zoom in, you can see consumer distribution in more detail. By default, the map displays consumer numbers at the division level, revealing that Dhaka has the highest count at 311k.

In [13]:

m.save('consumer_density_map.html')


If you want to save it as an image , it can be done in the following way <br>
But saving as an HTML file is more efficient, as the map is interactive

In [24]:
from selenium import webdriver

# Set up a headless browser
options = webdriver.ChromeOptions()
options.add_argument('headless')
browser = webdriver.Chrome(options=options)

# Open the HTML file in the browser
browser.get(r"file:///E:\Fordo pipeline\consumer_desnsity_map.html")

# Take a screenshot and save it as an image
browser.save_screenshot("consumer_density_map.png")

# Close the browser
browser.quit()
